# Graphical User Interface (GUI) to Model Solution in Hansen, Khorrami and Tourre

This notebook serves as a graphical user interface (GUI) for the framework developed in Hansen, Khorrami, and Tourre (forthcoming). You can use this notebook to solve and analyze models.

In [1]:
from IPython.display import Javascript
Javascript("Jupyter.notebook.execute_cells([3,4,5,6,7,8,9,14,15,16,17,18,19,20,21])")

<IPython.core.display.Javascript object>

In [14]:
%load_ext autoreload
%autoreload 2
%matplotlib inline 


## Load dependencies
from mfr.modelSoln import *
from mfr.jupyterWidgets import *
from IPython.core.display import display, HTML

display(HTML(
    '<script>'
        'var waitForPlotly = setInterval( function() {'
            'if( typeof(window.Plotly) !== "undefined" ){'
                'MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });'
                'MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);'
                'clearInterval(waitForPlotly);'
            '}}, 250 );'
    '</script>'
))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## Setting up parameters

First, configure the parameters. Afterwards, click button `Update parameters` and `Run model`.

In [16]:
display(paramsPanel)
#display(hhCap)
#out = widgets.interactive_output(displayHHProd, {'hhCap': hhCap})
#display(out)

def updateNShocks(*args):
    sigmas = [sigma_V_norm.value, sigma_Z_norm.value, sigma_Vtilde_norm.value]
    nShocks.value = len([s for s in sigmas if s > 0]) + 1
sigma_V_norm.observe(updateNShocks, 'value')
sigma_Z_norm.observe(updateNShocks, 'value')
sigma_Vtilde_norm.observe(updateNShocks, 'value')

out2 = widgets.interactive_output(displayShocks, {'nShocks': nShocks})
display(out2)

out3       = widgets.interactive_output(displayLoadSolution, {'loadSolution': loadSolution})
out4       = widgets.interactive_output(displayLoadParameters, {'loadParameters': loadParameters})
in_box     = VBox([widgets.Label(value="Input"), loadSolution, out3, loadParameters, out4], \
layout = Layout(width='90%'))
IOLine     = HBox([in_box, out_box])#, layout = box_layout)
display(IOLine)


Output()

After configuring parameters, click `Update parameters` and then `Run model` to solve the mode.

In [17]:
display(run_box)

## Model Results

### Plots

In [6]:
clear_output()
variableChoices   = defaultModel.label2Var.copy()
removeList        = ['Value Function of Experts','Value Function of Households', "Exogenous Growth", 
                     "Agg. Stochastic Variance", "Idio. Stochastic Variance", 
                     "Aggregate Consumption-to-Investment Ratio",
                     "Wealth Share"]
## Remove variables that are unnecessary to plot
for k in removeList:
    variableChoices.pop(k, None)

## Order the list of choices alphabetically
orderedList = []
for key in sorted(variableChoices.keys()):
    choice = (key, defaultModel.label2Var[key])
    orderedList.append(choice)
    
plotVar1.options  = orderedList
plotVar2.options  = orderedList
plotVar1.value    = 'chi()' ## choose default option

## Handle xaxis dropdown
stateVarOptions = []
for stateVar in defaultModel.stateVarList:
    stateVarOptions.append(defaultModel.stateVar2Label[stateVar])

xaxisStateDropdown.options = stateVarOptions

## Prepare ui
if defaultModel.params['nDims'] == 1:
    ui = widgets.HBox([plotVar1], layout = box_layout)
    plotVar1.description = 'Variable'
else:
    plotVar1.description           = 'Variable'
    xaxisStateDropdown.description = 'X-axis'
    ui = widgets.HBox([plotVar1, xaxisStateDropdown], layout = box_layout)
display(ui)

if defaultModel.params['nDims'] == 1:
    panel = widgets.interactive(defaultModel.plotPanel, varName1 = plotVar1, varName2 = plotVar2)
else:
    xaxisStateDropdown.value = 'Wealth Share'
    panel = widgets.interactive(defaultModel.plotPanel, varName1 = plotVar1, xaxisState = xaxisStateDropdown)

display(panel.children[-1])
plotVar1.value    = 'piE1()'


Output()

## Diagnostics

### Moments

In [18]:
variableChoices   = defaultModel.label2Var.copy()
removeList        = ['Value Function of Experts','Value Function of Households', "Drift of Exogenous Growth", 
                     "Drift of Agg. Stochastic Variance", "Drift of Idio. Stochastic Variance", "Aggregate Consumption-to-Investment Ratio", 
                     "Wealth Share Drift", "Wealth Share Diffusion: Coordinate 1"
                    , "Wealth Share Diffusion: Coordinate 2", "Wealth Share Diffusion: Coordinate 3"]
for k in removeList:
    try:
        variableChoices.pop(k, None)
    except:
        pass

momentsBox.options = variableChoices.keys()
display(momentsBox)

SelectMultiple(description='Equilibrium Quantities', layout=Layout(width='70%'), options=('Risk Price (Experts…

In [19]:
display(computeMomentsButton)

Button(description='Compute Moments', style=ButtonStyle())

In [20]:
display(computeCorrelationsButton)

Button(description='Compute Correlations', style=ButtonStyle())

### Shock Elasticities

In [21]:
display(displayShockElasButton)

Button(description='Display Shock Elasticities Panel', layout=Layout(width='50%'), style=ButtonStyle())

In [22]:
displayShockElas(defaultModel.stateVarList)

In [23]:
display(computeShockElasButton)

Button(description='Compute Shock Elasticities', layout=Layout(width='50%'), style=ButtonStyle())

In [13]:
displayShockElasPanel(defaultModel.stateVarList)
display(displayShockElasPanelButton)

Button(description='Display Shock Elasticities Plot', layout=Layout(width='50%'), style=ButtonStyle())